In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import arrow
import cmocean

In [2]:
%matplotlib inline

In [3]:
ds = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
ds

<xarray.Dataset>
Dimensions:    (gridX: 256, gridY: 266, time: 36408)
Coordinates:
  * time       (time) datetime64[ns] 2014-09-12 2014-09-12T01:00:00 ...
  * gridY      (gridY) float64 0.0 2.5e+03 5e+03 7.5e+03 1e+04 1.25e+04 ...
  * gridX      (gridX) float64 0.0 2.5e+03 5e+03 7.5e+03 1e+04 1.25e+04 ...
Data variables:
    atmpres    (time, gridY, gridX) float32 ...
    precip     (time, gridY, gridX) float32 ...
    qair       (time, gridY, gridX) float32 ...
    solar      (time, gridY, gridX) float32 ...
    tair       (time, gridY, gridX) float32 ...
    therm_rad  (time, gridY, gridX) float32 ...
    u_wind     (time, gridY, gridX) float32 ...
    v_wind     (time, gridY, gridX) float32 ...
Attributes:
    acknowledgement:           MEOPAR, ONC, Compute Canada, Environment Canada
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    coverage_content_type:     modelResult
    creator_email:             sallen@eos.ubc.ca
    creator_name: 

In [4]:
ds.time[0]

<xarray.DataArray 'time' ()>
array(1410480000000000000, dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2014-09-12
Attributes:
    _ChunkSizes:                 1
    _CoordinateAxisType:         Time
    actual_range:                [  1.41048000e+09   1.54154520e+09]
    axis:                        T
    ioos_category:               Time
    long_name:                   verification time generated by wgrib2 functi...
    reference_date:              2018.11.05 18:00:00 UTC
    reference_time:              1541440800.0
    reference_time_description:  kind of product unclear, reference date is v...
    reference_time_type:         0
    standard_name:               time
    time_origin:                 01-JAN-1970 00:00:00
    time_step:                   3600.0
    time_step_setting:           auto

In [5]:
ds.time[-1]

<xarray.DataArray 'time' ()>
array(1541545200000000000, dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2018-11-06T23:00:00
Attributes:
    _ChunkSizes:                 1
    _CoordinateAxisType:         Time
    actual_range:                [  1.41048000e+09   1.54154520e+09]
    axis:                        T
    ioos_category:               Time
    long_name:                   verification time generated by wgrib2 functi...
    reference_date:              2018.11.05 18:00:00 UTC
    reference_time:              1541440800.0
    reference_time_description:  kind of product unclear, reference date is v...
    reference_time_type:         0
    standard_name:               time
    time_origin:                 01-JAN-1970 00:00:00
    time_step:                   3600.0
    time_step_setting:           auto

In [9]:
test = ds.time.sel[time = "2015-01-01":"2015-03-31"]

SyntaxError: invalid syntax (<ipython-input-9-ebb0784b7534>, line 1)